In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst

#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/pyfeng')
import pyfeng as pf

# Garch Diffusion Model

* Barone-Adesi G, Rasmussen H, Ravanelli C (2005) An option pricing formula for the GARCH diffusion model. Computational Statistics & Data Analysis 49:287–310. https://doi.org/10.1016/j.csda.2004.05.014
* `PyFENG` implements the second order ($p^{gd(2)}$) solution.

### The implementations are based on the following student course projects

https://github.com/PyFE/pyfedev/blob/master/test/Test_Garch.ipynb

https://github.com/PanyuLi/PHBS_ASP_Project/blob/main/Test_CondMC_OUSV_Garch.ipynb

## Run only one of the tables below

## Table 1

In [3]:
mr, vov, sigma = 18, 1.8, 0.16/18

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    0.0008, 0.0803, 1.2918, 5.1002, 10.0023, 
    0.0184, 0.2995, 1.8283, 5.3499, 10.0365, 
    0.0665, 0.5403, 2.2409, 5.6156, 10.1133, 
    0.1395, 0.7761, 2.5889, 5.8722, 10.2189, 
    0.3296, 1.2163, 3.1726, 6.3464, 10.4735, 
    0.593, 1.6921, 3.7551, 6.8551, 10.8063, 
    1.544, 3.0579, 5.3118, 8.3054, 11.9445
]
p_grid = np.array(p_grid).reshape((-1,5))
p_paper = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

## Table 2

In [4]:
mr, vov, sigma = 29.23, 3.65, 0.53/29.23

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    0.0242, 0.3107, 1.8317, 5.3614, 10.0449, 
    0.1523, 0.7902, 2.5995, 5.8868, 10.2343, 
    0.3461, 1.2347, 3.1904, 6.3657, 10.4926, 
    0.5675, 1.6393, 3.6883, 6.7985, 10.773, 
    1.0316, 2.3552, 4.5229, 7.5603, 11.3378, 
    1.5781, 3.0983, 5.3551, 8.3481, 11.9835, 
    3.2723, 5.1536, 7.5763, 10.5196, 13.9318
]
p_grid = np.array(p_grid).reshape((-1,5))
p_paper = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

## Table 3

In [5]:
mr, vov, sigma = 2, 0.8, 0.18/2

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    0.7902, 2.0008, 4.1164, 7.1837, 11.0473, 
    1.9061, 3.5118, 5.8074, 8.7855, 12.3642, 
    2.8977, 4.7092, 7.0994, 10.0503, 13.5033, 
    3.7841, 5.7309, 8.1859, 11.1281, 14.5104, 
    5.3346, 7.4598, 10.0057, 12.9502, 16.2582, 
    6.9328, 9.1984, 11.8214, 14.7811, 18.0493, 
    11.3323, 13.8771, 16.6736, 19.7049, 22.9531
]
p_grid = np.array(p_grid).reshape((-1,5))
p_paper = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

## Table 4

In [6]:
mr, vov, sigma = 2, 1.2, 0.18/2

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    0.7905, 1.9913, 4.1018, 7.1733, 11.0458, 
    1.893, 3.4821, 5.7705, 8.7537, 12.3461, 
    2.8658, 4.657, 7.0387, 9.9948, 13.4635, 
    3.7322, 5.6565, 8.1021, 11.0492, 14.4479, 
    5.2445, 7.3456, 9.881, 12.8294, 16.1533, 
    6.8054, 9.0469, 11.659, 14.6211, 17.9033, 
    11.1406, 13.6659, 16.4522, 19.4824, 22.7376
]
p_grid = np.array(p_grid).reshape((-1,5))
p_paper = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

## Table 5

In [7]:
mr, vov, sigma = 4, 1.2, 0.09/4

strike = np.arange(90,111,5) # strike price
spot = 100 # spot asset price
texp = np.array([30, 60, 90, 120, 180, 252, 504])/252  # time to maturity

p_grid = [
    0.0419, 0.4273, 2.0536, 5.4913, 10.0747, 
    0.2415, 1.0085, 2.8961, 6.1225, 10.355, 
    0.5073, 1.5223, 3.5421, 6.6732, 10.6955, 
    0.7907, 1.9823, 4.0879, 7.1634, 11.0443, 
    1.3566, 2.7896, 5.0069, 8.0206, 11.7202, 
    2.0028, 3.6251, 5.9284, 8.9051, 12.4745, 
    3.9603, 5.9329, 8.4004, 11.3412, 14.7103
]
p_grid = np.array(p_grid).reshape((-1,5))
p_paper = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

## PyFeng Implementation (2nd order: $p^{gd(2)}$)

In [8]:
m1 = pf.GarchUncorrBaroneAdesi2004(sigma, vov=vov, mr=mr, theta=sigma)
p_grid = m1.price(strike, spot, texp[:, None], cp=-1)

p_pyfeng = pd.DataFrame(data=p_grid, columns=strike, index=np.int32(texp*252))

In [9]:
# Difference between the paper and pyfeng values
p_pyfeng.round(4) - p_paper

,90,95,100,105,110
30,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0
90,0.0,0.0,0.0,0.0,0.0
120,0.0,0.0,0.0,0.0,0.0
180,0.0,0.0,0.0,0.0,0.0
252,0.0,0.0,0.0,0.0,0.0
504,0.0,0.0,0.0,0.0,0.0


## Monte-Carlo with Milstein Scheme 

In [10]:
m2 = pf.GarchMcTimeDisc(sigma, vov=vov, mr=mr, theta=sigma)
m2.set_num_params(n_path=1e5, dt=1/50, rn_seed=123456, scheme=1)  #, scheme=1

## For Table 1 and 3, dt = 1/50 is enough
## For Table 2, dt = 1/100 to avoid zero v_t

In [11]:
p_grid = np.zeros((len(texp), len(strike)))
for i, t1 in enumerate(texp):
    p_grid[i, :] = m2.price(strike, spot, t1, cp=-1)
p_mc = pd.DataFrame(data=p_grid, index=np.int32(texp*252), columns=strike)

In [12]:
# Difference between the paper and MC method values
p_mc.round(4) - p_paper

,90,95,100,105,110
30,0.0001,0.0000,-0.0001,0.0000,0.0001
60,-0.0006,0.0000,0.0006,0.0001,-0.0007
90,-0.0017,-0.0002,0.0008,-0.0002,-0.0017
120,-0.0016,0.0008,0.0021,0.0010,-0.0014
180,-0.0009,0.0020,0.0034,0.0023,-0.0004
252,0.0018,0.0047,0.0059,0.0049,0.0027
504,0.0029,0.0042,0.0048,0.0045,0.0035
